In [1]:
# --------------------------------------------------------
# Script 1: Fetch Yahoo Finance beta for a list of tickers
# --------------------------------------------------------
# pip install yfinance
import yfinance as yf
import pandas as pd

def get_yahoo_betas(tickers):
    """
    Fetch 'beta' as reported by Yahoo Finance for each ticker.

    Parameters
    ----------
    tickers : list of str
        e.g. ["AAPL", "MSFT", "TSLA"]

    Returns
    -------
    pandas.Series
        Index = ticker, value = beta (float or NaN)
    """
    betas = {}
    for t in tickers:
        try:
            ticker = yf.Ticker(t)
            info = ticker.info  # may be slow the first time
            beta = info.get("beta", None)
            betas[t] = beta
        except Exception as e:
            print(f"Error fetching {t}: {e}")
            betas[t] = None

    return pd.Series(betas, name="Yahoo_Beta")


if __name__ == "__main__":
    tickers = ["AAPL", "MSFT", "TSLA", "JPM", "PG"]
    yahoo_betas = get_yahoo_betas(tickers)
    print("\nYahoo Finance betas:")
    print(yahoo_betas)



Yahoo Finance betas:
AAPL    1.107
MSFT    1.070
TSLA    1.878
JPM     1.075
PG      0.382
Name: Yahoo_Beta, dtype: float64


In [2]:
# --------------------------------------------------------
# Script 2: Compute beta yourself from price history
# --------------------------------------------------------
# pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

def download_price_data(tickers, start="2015-01-01", end=None):
    """
    Download Adjusted Close prices for a list of tickers using yfinance.

    Parameters
    ----------
    tickers : list of str
    start   : str (YYYY-MM-DD)
    end     : str or None (default: today)

    Returns
    -------
    prices : DataFrame
        Columns = tickers, Index = dates
    """
    if end is None:
        end = datetime.today().strftime("%Y-%m-%d")

    data = yf.download(tickers, start=start, end=end, auto_adjust=False, progress=False)
    # yfinance returns MultiIndex columns if multiple tickers; we want Adj Close
    if isinstance(data.columns, pd.MultiIndex):
        prices = data["Adj Close"]
    else:
        # single ticker case
        prices = data["Adj Close"].to_frame()
        prices.columns = tickers

    return prices


def compute_returns(prices, freq="D"):
    """
    Compute simple returns from price data.

    Parameters
    ----------
    prices : DataFrame
        Adjusted close prices
    freq   : str
        "D" daily, "W" weekly, "M" monthly (we'll just resample if needed)

    Returns
    -------
    returns : DataFrame
        Return series at chosen frequency
    """
    if freq.upper() == "D":
        px = prices
    elif freq.upper() == "W":
        px = prices.resample("W-FRI").last()
    elif freq.upper() == "M":
        px = prices.resample("M").last()
    else:
        raise ValueError("freq must be 'D', 'W', or 'M'")

    returns = px.pct_change().dropna()
    return returns


def compute_betas(returns, market_col):
    """
    Compute beta of each column relative to a chosen market column.

    Parameters
    ----------
    returns    : DataFrame
        Asset returns. Columns include market_col and other tickers.
    market_col : str
        Column name for the market benchmark (e.g. '^GSPC' or 'SPY')

    Returns
    -------
    betas : Series
        Beta of each asset vs the market (market beta should be ~1).
    """
    market = returns[market_col]
    betas = {}

    # Var of market
    var_m = np.var(market, ddof=1)

    for col in returns.columns:
        if col == market_col:
            continue
        asset = returns[col]
        cov_im = np.cov(asset, market, ddof=1)[0, 1]
        beta_i = cov_im / var_m
        betas[col] = beta_i

    return pd.Series(betas, name=f"Beta_vs_{market_col}")


if __name__ == "__main__":
    # Example usage
    tickers = ["AAPL", "MSFT", "TSLA", "JPM", "PG"]
    market_ticker = "^GSPC"  # S&P 500

    all_tickers = tickers + [market_ticker]

    # 1) Download price data
    prices = download_price_data(all_tickers, start="2018-01-01")

    # 2) Compute returns (daily here; you can switch to "W" or "M")
    returns = compute_returns(prices, freq="D")

    # 3) Compute betas vs the chosen market
    betas_custom = compute_betas(returns, market_col=market_ticker)

    print("\nCustom-computed betas vs S&P 500:")
    print(betas_custom)



Custom-computed betas vs S&P 500:
AAPL    1.217118
JPM     1.061754
MSFT    1.177855
PG      0.495514
TSLA    1.632353
Name: Beta_vs_^GSPC, dtype: float64


In [3]:
from sklearn.linear_model import LinearRegression

def regression_beta(returns, market_col):
    """
    Compute alpha and beta via linear regression:
        R_i = alpha_i + beta_i * R_m + error

    Returns
    -------
    DataFrame with columns ['alpha', 'beta'] for each asset.
    """
    market = returns[market_col].values.reshape(-1, 1)
    results = {}

    for col in returns.columns:
        if col == market_col:
            continue

        asset = returns[col].values.reshape(-1, 1)
        reg = LinearRegression().fit(market, asset)
        alpha = reg.intercept_[0]
        beta = reg.coef_[0][0]
        results[col] = {"alpha": alpha, "beta": beta}

    return pd.DataFrame(results).T  # assets as rows


if __name__ == "__main__":
    # continuing from above
    reg_results = regression_beta(returns, market_col=market_ticker)
    print("\nRegression alpha & beta vs S&P 500:")
    print(reg_results)



Regression alpha & beta vs S&P 500:
         alpha      beta
AAPL  0.000494  1.217118
JPM   0.000234  1.061754
MSFT  0.000425  1.177855
PG    0.000145  0.495514
TSLA  0.001448  1.632353


In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# --------------------------------------------------------
# Helpers
# --------------------------------------------------------

def download_prices(tickers, start, end=None):
    """
    Download Adjusted Close prices for given tickers.
    """
    if end is None:
        end = datetime.today().strftime("%Y-%m-%d")

    data = yf.download(tickers, start=start, end=end, auto_adjust=False, progress=False)

    # yfinance returns MultiIndex columns if multiple tickers; we want 'Adj Close'
    if isinstance(data.columns, pd.MultiIndex):
        prices = data["Adj Close"]
    else:
        prices = data["Adj Close"].to_frame()
        prices.columns = tickers

    return prices


def compute_returns(prices, freq="D"):
    """
    Convert price series to return series at the requested frequency.

    freq:
        "D" -> daily
        "W" -> weekly (Friday close)
        "M" -> monthly (month-end close)
    """
    if freq.upper() == "D":
        px = prices
    elif freq.upper() == "W":
        px = prices.resample("W-FRI").last()
    elif freq.upper() == "M":
        px = prices.resample("M").last()
    else:
        raise ValueError("freq must be 'D', 'W', or 'M'")

    rets = px.pct_change().dropna()
    return rets


def compute_beta(returns, asset_col, market_col):
    """
    Compute CAPM beta of asset_col vs market_col using:
        beta = Cov(R_i, R_m) / Var(R_m)
    """
    r_i = returns[asset_col]
    r_m = returns[market_col]

    cov_im = np.cov(r_i, r_m, ddof=1)[0, 1]
    var_m  = np.var(r_m, ddof=1)

    beta = cov_im / var_m
    return beta


# --------------------------------------------------------
# Main: compare 5Y daily, 2Y weekly, 1Y monthly betas
# --------------------------------------------------------
if __name__ == "__main__":

    asset_ticker = "AAPL"      # change to any ticker you like
    market_ticker = "^GSPC"    # S&P 500 as market

    today = datetime.today()

    # 1) 5-year DAILY beta
    start_5y = (today - timedelta(days=5*365)).strftime("%Y-%m-%d")
    prices_5y = download_prices([asset_ticker, market_ticker], start=start_5y)
    rets_5y_daily = compute_returns(prices_5y, freq="D")
    beta_5y_daily = compute_beta(rets_5y_daily, asset_ticker, market_ticker)

    # 2) 2-year WEEKLY beta
    start_2y = (today - timedelta(days=2*365)).strftime("%Y-%m-%d")
    prices_2y = download_prices([asset_ticker, market_ticker], start=start_2y)
    rets_2y_weekly = compute_returns(prices_2y, freq="W")
    beta_2y_weekly = compute_beta(rets_2y_weekly, asset_ticker, market_ticker)

    # 3) 1-year MONTHLY beta
    start_1y = (today - timedelta(days=365)).strftime("%Y-%m-%d")
    prices_1y = download_prices([asset_ticker, market_ticker], start=start_1y)
    rets_1y_monthly = compute_returns(prices_1y, freq="M")
    beta_1y_monthly = compute_beta(rets_1y_monthly, asset_ticker, market_ticker)

    # ----------------------------------------------------
    # Print comparison
    # ----------------------------------------------------
    print(f"\nBeta estimates for {asset_ticker} vs {market_ticker}:\n")
    print(f"5-year  DAILY   beta : {beta_5y_daily:.3f}")
    print(f"2-year  WEEKLY  beta : {beta_2y_weekly:.3f}")
    print(f"1-year  MONTHLY beta : {beta_1y_monthly:.3f}\n")



Beta estimates for AAPL vs ^GSPC:

5-year  DAILY   beta : 1.252
2-year  WEEKLY  beta : 1.126
1-year  MONTHLY beta : 0.589



/var/folders/7j/8pz9dqx16j514s66gfbnpf1r0000gn/T/ipykernel_59587/3140839221.py:43: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  px = prices.resample("M").last()
